**Вызов библиотек**

In [8]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

**Подготовка формул(среднее значение, матожидание) для алгоритма детектирования**

In [9]:
nabor100 = []
u_mas = np.zeros((480,640,1))

In [10]:
def param(nabor):
    u = np.mean(nabor, axis=0)
    q = np.var(nabor, axis=0) #std
    
    return u, q

In [11]:
def obnovlenie(u , q, frame, lr):
    n_u = lr * u + (1 - lr) * frame
    n_q = lr * (np.square(q) + np.square(n_u - u)) + (1 - lr) * np.square(frame - n_u)
    return n_u, np.sqrt(n_q)

**Алгоритм детектирования**

In [12]:
vid_capture = cv2.VideoCapture('C:/Users/igor1/LABA3OPENCV/1.mp4')
if (vid_capture.isOpened() == False):
    print("Ошибка открытия видеофайла")
# Чтение fps и количества кадров
else:
    # Получить информацию о частоте кадров
    # Можно заменить 5 на CAP_PROP_FPS, это перечисления
    fps = vid_capture.get(5)
    print('Фреймов в секунду: ', fps,'FPS')

nabor100 = []
iter = 3400
vid_capture.isOpened()
flag = 0
for i in range(iter):
    x, background = vid_capture.read()
    background_gray = cv2.cvtColor(background[:, 90:570], cv2.COLOR_BGR2GRAY)
    if iter > 0 and len(nabor100) < 200:
        nabor100.append(background_gray)
    
u, q = param(nabor100)    
while(True):
    ret, frame = vid_capture.read()
    if ret == True:
        frame_gray = cv2.cvtColor(frame[:, 90:570], cv2.COLOR_BGR2GRAY)
        p =  1 / (q * np.sqrt(2 * np.pi)) * np.exp(-np.square(frame_gray - u) / (2 * q ** 2))
        pp = p.copy()
        p[p < 0.0061] = 0  
        pp[pp < 0.01] = 255   
        grad_y = cv2.Sobel(p, ddepth=cv2.CV_64F, dx = 1, dy = 1, ksize=17)
        kernel = np.ones((3,3),np.uint8)
        erosion = cv2.morphologyEx(grad_y, cv2.MORPH_CLOSE, kernel)
        left_up_corner = (0, 0)
        right_low_corner = (0, 0)
        
        cv2.imwrite('C:/Users/igor1/LABA3OPENCV/rezervuar/erosion.png', erosion)
        erosion_x = cv2.imread('C:/Users/igor1/LABA3OPENCV/rezervuar/erosion.png')
        erosion_x = cv2.cvtColor(erosion_x, cv2.COLOR_BGR2GRAY)
        
        erosion_x = cv2.resize(erosion_x, (480,480))
        
        window= 24
        
        for i in range(0,erosion_x.shape[0], 24):
            ee = []
            for j in range(0, erosion_x.shape[1], 24):
                if np.count_nonzero(erosion_x[i:i+window,j:j+window]) > 300:
                    cv2.rectangle(frame[:, 90:570], ((j - 25), (i - 25)), ((j+2*window + 25), (i+2*window + 25)), (0,255,0), 1)
                    ee.append([j - 25, i - 25, j+2*window + 25, i+2*window + 25])
                    eee = np.array(ee)
                elif len(eee.shape) > 1:
                    if len(eee.shape) > 1:
                        eee_sred = np.mean(eee, axis = 0)
                    j_max = int(eee_sred[0]) 
                    i_max = int(eee_sred[1])  
                    jj_max = int(eee_sred[2])    
                    ii_max = int(eee_sred[3])    
        cv2.rectangle(frame[:, 90:570], ((j_max - 30), (i_max - 30)), ((jj_max + 30), (ii_max + 30)), (255,255,0), 1)     
        cv2.imshow('Look_1', p)
        cv2.imshow('Look_2', erosion_x)
        cv2.imshow('Look_3', frame[:, 90:570])
        
        u, q = obnovlenie(u, q, frame_gray, 0.05)

        flag = flag + 1
        key = cv2.waitKey(20)
        if (key == ord('q')) or key == 27:
            break
    else:
        break
vid_capture.release()
cv2.destroyAllWindows()

Фреймов в секунду:  20.00329218106996 FPS
